In [ ]:
from fhir.resources.bundle import Bundle 
import json
import jmespath
from kafka import KafkaProducer
import glob

In [ ]:
!docker compose stop
!docker compose rm -f
!docker compose up -d

In [ ]:
def load_fhir_resources(f):
    with open(f, "r") as file:
        data = json.load(file)
    resources = jmespath.search("entry[*]", data)
    return resources
def send_resources_to_kafka(resources):
    producer = KafkaProducer(bootstrap_servers='localhost:9095', value_serializer=lambda v: json.dumps(v).encode('utf-8'))
    for resource in resources:
        if "fullUrl" in resource:
            print("{} - {} ".format(resource["resource"]["resourceType"], resource["fullUrl"]))
        else:
            print(resource["id"])
        producer.send(resource["resource"]["resourceType"], key=resource["fullUrl"].encode(), value=resource["resource"])
    print("waiting for flush")
    producer.flush()    

## Patient

In [ ]:
for f in glob.glob("output/fhir/*"):
    if ("hospitalInformation" not in f) and ("practitionerInformation" not in f): 
        print(f"Loading {f}")
        resources = load_fhir_resources(f)
        send_resources_to_kafka(resources)

## Practioner

In [ ]:
for f in glob.glob("output/fhir/*"):
    if ("practitionerInformation"  in f): 
        print(f"Loading {f}")
        resources = load_fhir_resources(f)
        send_resources_to_kafka(resources)

## Hospital

In [ ]:
for f in glob.glob("output/fhir/*"):
    if ("hospitalInformation"  in f): 
        print(f"Loading {f}")
        resources = load_fhir_resources(f)
        send_resources_to_kafka(resources)